# Week 12 Coding Homework Notebook


In [2]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 40.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

np.random.seed(0)
plt.rcParams['figure.figsize'] = (8, 5)


## Question 1 

Find the effect in dataset 12.1, given that Group = 1 is the treatment group and Time > 0 is the treatment time. 



In [4]:
data_path_q1 = 'homework_12.1.csv'

df1 = pd.read_csv(data_path_q1)
print('Data 12.1 shape:', df1.shape)
display(df1.head())

df1['treat'] = (df1['Group'] == 1).astype(int)
df1['post'] = (df1['Time'] > 0).astype(int)

group_means = (
    df1
    .groupby(['treat', 'post'])['Y']
    .mean()
    .unstack('post')
)
group_means.columns = ['pre', 'post']  # 0 = pre, 1 = post
display(group_means)

treat_change = group_means.loc[1, 'post'] - group_means.loc[1, 'pre']
control_change = group_means.loc[0, 'post'] - group_means.loc[0, 'pre']
did_effect = treat_change - control_change

print('Treatment group change:', treat_change)
print('Control group change:  ', control_change)
print('DID effect (manual):   ', did_effect)

model_q1 = ols('Y ~ treat + post + treat:post', data=df1).fit()
print(model_q1.summary())
print('\nDID effect from regression (treat:post coefficient):', model_q1.params['treat:post'])


Data 12.1 shape: (10000, 4)


,Unnamed: 0,Y,Time,Group
0,0,-0.232900,-1.193204,0
1,1,2.848846,-1.607748,1
2,2,0.550209,-0.269793,0
3,3,2.198280,7.743730,0
4,4,4.111044,-4.244359,1


,pre,post
treat,,
0,0.980217,2.493206
1,3.022227,5.479226


Treatment group change: 2.4569993470017906
Control group change:   1.5129889686488909
DID effect (manual):    0.9440103783528997
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     5866.
Date:                Sun, 07 Dec 2025   Prob (F-statistic):               0.00
Time:                        04:44:44   Log-Likelihood:                -14214.
No. Observations:               10000   AIC:                         2.844e+04
Df Residuals:                    9996   BIC:                         2.846e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--

## Question 2 

With dataset 12.2, run a test to check prior trends. To do this, run a linear regression on the data before Time = 0, with an interaction term equal to Group x Time. Which is the closest to the t-value of the interaction term? 


In [5]:
data_path_q2 = 'homework_12.2.csv'

df2 = pd.read_csv(data_path_q2)
print('Data 12.2 shape:', df2.shape)
display(df2.head())

df2_pre = df2[df2['Time'] < 0].copy()
print('Pre-treatment subset shape:', df2_pre.shape)

df2_pre['Group'] = df2_pre['Group'].astype(int)

model_q2 = ols('Y ~ Time * Group', data=df2_pre).fit()
print(model_q2.summary())

t_interaction = model_q2.tvalues['Time:Group']
print('\nT-value for Group × Time interaction:', t_interaction)


Data 12.2 shape: (20000, 4)


,Unnamed: 0,Y,Time,Group
0,0,0.283614,-3.614416,0
1,1,3.181495,-3.064196,0
2,2,1.401619,-4.109261,0
3,3,2.019632,-0.706696,1
4,4,1.032628,-1.806227,0


Pre-treatment subset shape: (9944, 4)
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     1591.
Date:                Sun, 07 Dec 2025   Prob (F-statistic):               0.00
Time:                        04:46:30   Log-Likelihood:                -13981.
No. Observations:                9944   AIC:                         2.797e+04
Df Residuals:                    9940   BIC:                         2.800e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     

## Question 3 
Question 3
10
 Points
Question 3
With the following, make an error such that the correlation (np.corrcoef) between the error at time t and time t + 1 is 0.80 on average, the mean error is 0, and the standard deviation of the error is 1. (These may be approximate.) There are 10,000 items. Given: 

X = this error 

Y = 2 * X + this error (recalculated with new randomness relative to the previous line) 



The standard error of the estimated X coefficient computed via Ordinary Least Squares (OLS) basic standard error should be about 0.01. Then the standard error computed via simulation (the standard deviation of the estimated coefficient over many trials) is closest to: 



Note: you might consider Googling / asking ChatGPT how to generate an autocorrelated time series with a specific correlation and standard deviation. 



In [6]:
def generate_ar1_errors(n=10000, rho=0.8):
    """Generate AR(1) errors with correlation rho and approx sd=1."""
    eps_sd = np.sqrt(1 - rho**2)  # so that stationary variance is 1
    eps = np.random.normal(0, eps_sd, size=n)
    e = np.zeros(n)
    for t in range(1, n):
        e[t] = rho * e[t-1] + eps[t]
    return e

n = 10000
rho_target = 0.8
e = generate_ar1_errors(n=n, rho=rho_target)
print('Mean error:', np.mean(e))
print('Std error: ', np.std(e))
print('Lag-1 correlation (e_t, e_{t+1}):', np.corrcoef(e[:-1], e[1:])[0, 1])

X = e
new_error = generate_ar1_errors(n=n, rho=rho_target)
Y = 2 * X + new_error

X_mat = sm.add_constant(X)
ols_res = sm.OLS(Y, X_mat).fit()
beta_hat = ols_res.params[1]
se_naive = ols_res.bse[1]

print('\nSingle-run OLS estimate of beta:', beta_hat)
print('Naive OLS standard error:        ', se_naive)

R = 200  # you can increase this if you have time
beta_hats = []
for r in range(R):
    e = generate_ar1_errors(n=n, rho=rho_target)
    X = e
    new_error = generate_ar1_errors(n=n, rho=rho_target)
    Y = 2 * X + new_error
    X_mat = sm.add_constant(X)
    res = sm.OLS(Y, X_mat).fit()
    beta_hats.append(res.params[1])

beta_hats = np.array(beta_hats)
sim_sd = beta_hats.std(ddof=1)

print('\nSimulation results (R =', R, ')')
print('Mean of beta_hat over simulations:', beta_hats.mean())
print('SD of beta_hat over simulations:  ', sim_sd)
print('\nCompare: naive OLS SE vs simulation SD')
print('Naive OLS SE (single run):', se_naive)
print('Simulation SD:            ', sim_sd)


Mean error: -0.056218435020453435
Std error:  0.978853495984585
Lag-1 correlation (e_t, e_{t+1}): 0.7960555513366709

Single-run OLS estimate of beta: 1.9869683638608868
Naive OLS standard error:         0.010193545837902288

Simulation results (R = 200 )
Mean of beta_hat over simulations: 2.000489308914232
SD of beta_hat over simulations:   0.020704965310920153

Compare: naive OLS SE vs simulation SD
Naive OLS SE (single run): 0.010193545837902288
Simulation SD:             0.020704965310920153


## Question 4 
Find the variance inflation factor of X1: 

np.random.seed(0) 

X1 = np.random.normal(0, 1, 1000) 

X2 = np.random.normal(0, 1, 1000) + X1 

X3 = np.random.normal(0, 1, 1000) + 2 * X2 



In [8]:
np.random.seed(0)
X1 = np.random.normal(0, 1, 1000)
X2 = np.random.normal(0, 1, 1000) + X1
X3 = np.random.normal(0, 1, 1000) + 2 * X2

df_vif = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3})

X_mat = sm.add_constant(df_vif[['X2', 'X3']])
res_vif = sm.OLS(df_vif['X1'], X_mat).fit()
R2 = res_vif.rsquared
vif_X1 = 1 / (1 - R2)

print(res_vif.summary())
print('\nR^2 for regression of X1 on X2 and X3:', R2)
print('VIF for X1:', vif_X1)


                            OLS Regression Results                            
Dep. Variable:                     X1   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     487.7
Date:                Sun, 07 Dec 2025   Prob (F-statistic):          1.95e-148
Time:                        04:50:25   Log-Likelihood:                -1064.7
No. Observations:                1000   AIC:                             2135.
Df Residuals:                     997   BIC:                             2150.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0301      0.022     -1.352      0.1

## REFLECTION


## Question
Construct a dataset in which prior trends do not hold, and in which this makes the differences-in-differences come out wrong.  Explain why the
differences-in-differences estimate of the effect comes out higher or lower than the actual effect.


For this reflection, I constructed a dataset in which the treatment and control groups followed different trends before the treatment period, violating the parallel trends assumption required for difference-in-differences. Specifically, the treatment group had a steeper upward trend than the control group even before the treatment occurred, and I then introduced a treatment effect at a later time. When I applied a differences-in-differences estimator, the estimated treatment effect was larger than the true effect built into the data. This happened because DID attributed not only the actual treatment effect, but also the pre-existing difference in trends, to the treatment itself. Since the treatment group was already improving faster than the control group prior to treatment, the DID estimator overstated the effect by confusing the continuation of the pre-trend with the causal impact of the treatment. This demonstrates why checking for parallel pre-treatment trends is essential before interpreting a DID estimate as causal.

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(0)

# Number of observations per (group, time) cell
n_per_cell = 100

# Time periods and groups
times = np.arange(-5, 5)        # -5, -4, ..., 4
groups = np.array([0, 1])       # 0 = control, 1 = treatment

# Create all (time, group) combinations and repeat within each cell
time_grid, group_grid = np.meshgrid(times, groups, indexing='ij')
time_vec = np.repeat(time_grid.ravel(), n_per_cell)
group_vec = np.repeat(group_grid.ravel(), n_per_cell)

# Indicator for post-treatment period
post = (time_vec >= 0).astype(int)

# Different pre-trends: treatment group has a steeper slope
control_trend = 0.5 * time_vec
treat_trend   = 1.0 * time_vec   # steeper line

trend = np.where(group_vec == 0, control_trend, treat_trend)

# True treatment effect
true_effect = 2.0

# Outcome with noise
noise = np.random.normal(0, 1, len(time_vec))
Y = trend + true_effect * post * group_vec + noise

# Put into a DataFrame
df = pd.DataFrame({
    "Y": Y,
    "Time": time_vec,
    "Group": group_vec,
    "Post": post
})

# DID regression: Y ~ Group + Post + Group*Post
df["interaction"] = df["Group"] * df["Post"]
X = sm.add_constant(df[["Group", "Post", "interaction"]])
res = sm.OLS(df["Y"], X).fit()

print(res.summary())
print("True treatment effect:", true_effect)
print("Estimated DID effect:", res.params["interaction"])


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     2220.
Date:                Sun, 07 Dec 2025   Prob (F-statistic):               0.00
Time:                        05:35:02   Log-Likelihood:                -3612.9
No. Observations:                2000   AIC:                             7234.
Df Residuals:                    1996   BIC:                             7256.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.5350      0.066    -23.274      